In [1]:
puzzle = '''jio a, +18
inc a
tpl a
inc a
tpl a
tpl a
tpl a
inc a
tpl a
inc a
tpl a
inc a
inc a
tpl a
tpl a
tpl a
inc a
jmp +22
tpl a
inc a
tpl a
inc a
inc a
tpl a
inc a
tpl a
inc a
inc a
tpl a
tpl a
inc a
inc a
tpl a
inc a
inc a
tpl a
inc a
inc a
tpl a
jio a, +8
inc b
jie a, +4
tpl a
inc a
jmp +2
hlf a
jmp -7'''.splitlines()

In [16]:
import re
instruction_matcher = re.compile(r'((?P<jtype>j\w{2})( (?P<jreg>.),)? (?P<joffset>[+-]\d+))|((?P<mtype>inc|tpl|hlf) (?P<mreg>.))')

In [38]:
class Computer:
    def __init__(self, instructions, a=0, b=0):
        self.instructions = instructions
        self.index = 0
        self.a = a
        self.b = b
    def run(self):
        while self.index < len(self.instructions):
            current = self.instructions[self.index]
            instruction = instruction_matcher.match(current).groupdict()
            jtype = instruction['jtype']
            jreg = instruction['jreg']
            mtype = instruction['mtype']
            mreg = instruction['mreg']
            if jtype is not None: # This is a jump instruction
                if any([
                        jtype == 'jmp',
                        jtype == 'jio' and int(getattr(self, str(instruction['jreg']), -1)) == 1,
                        jtype == 'jie' and int(getattr(self, str(instruction['jreg']), -1)) % 2 == 0]):
                    self.index += int(instruction['joffset'])
                    continue
            else: # This is a modify instruction
                mcond = {
                    'hlf': lambda r: r//2,
                    'tpl': lambda r: r*3,
                    'inc': lambda r: r+1,
                }
                setattr(self, mreg, mcond[mtype](getattr(self, mreg)))
            self.index += 1
        return self.a, self.b

In [39]:
comp = Computer(puzzle)
comp.run()

(1, 307)

In [40]:
comp2 = Computer(puzzle, a=1)
comp2.run()

(1, 160)